# import

In [59]:
import torch
import torch.nn as nn
from vilt.modules import heads, objectives
import vilt.modules.vision_transformer as vit
import torch.nn.functional as F
import random
from typing import OrderedDict
import os
import pandas as pd
import numpy as np
from vilt.transforms import pixelbert_transform
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
import gc
import torch.optim as optim
from torch.optim import lr_scheduler
from collections import defaultdict
import wandb

from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold


# config

In [60]:


class config:
    debug = False
    exp_name = "vilt"
    seed = 101
    batch_size = 4096  # this is a desired batch size; pl trainer will accumulate gradients when per step batch is smaller.
    train_batch_size = 32
    valid_batch_size = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # root_path = r'E:\\Download\\xiangguan' # 存放数据的根目录
    root_path = r'/home/junsheng/data/xiangguan' # 存放数据的根目录
    n_fold = 5

    # wandb 
    wandb_name = "vilt|290 sensor only"
    

    # Image setting
    train_transform_keys = ["pixelbert"]
    val_transform_keys = ["pixelbert"]
    img_size = 384
    max_image_len = -1
    patch_size = 32
    draw_false_image = 1
    image_only = False

    # Sensor
    # senser_input_num = 11 # 翔冠的传感器参数
    senser_input_num = 19 # 天航的传感器参数
    
    # Text Setting
    vqav2_label_size = 3129
    max_text_len = 40
    tokenizer = "bert-base-uncased"
    vocab_size = 30522 # vocabulary词汇数量
    whole_word_masking = False
    mlm_prob = 0.15
    draw_false_text = 0

    # Transformer Setting
    vit = "vit_base_patch32_384"
    hidden_size = 768  # 嵌入向量大小
    num_heads = 12
    num_layers = 12
    mlp_ratio = 4
    drop_rate = 0.1

    # Optimizer Setting
    optim_type = "adamw"
    learning_rate = 1e-3
    weight_decay = 1e-4 # 0.01 ->1e-4
    decay_power = 1
    max_epoch = 50
    max_steps = 25000
    warmup_steps = 2500
    end_lr = 0
    lr_mult = 1  # multiply lr for downstream heads
    # T_max = 8000/train_batch_size*max_epoch 
    T_max = 1000/train_batch_size*max_epoch 

    # Downstream Setting
    get_recall_metric = False


    # below params varies with the environment
    data_root = ""
    log_dir = "result"
    per_gpu_batchsize = 0  # you should define this manually with per_gpu_batch_size=#
    num_gpus = 1
    num_nodes = 1
    load_path = "weights/vilt_200k_mlm_itm.ckpt"
    # load_path = "save_model_dict.pt"
    num_workers = 1
    precision = 16

# config = vars(config)
# config = dict(config)
config

if config.debug:
    config.max_epoch = 5

In [61]:
def setup_seed(seed):

    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    # torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
setup_seed(config.seed)

# wandb

In [62]:
os.environ["WANDB_MODE"] = 'dryrun' # 离线模式
try:
    # wandb.log(key="*******") # if debug
    wandb.login() # storage in ~/.netrc file
    anonymous = None
except:
    anonymous = "must"
    print('\nGet your W&B access token from here: https://wandb.ai/authorize\n')


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# 数据

In [63]:
df_tianhang = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-rice.csv")
df_tianhang['image_path'] = df_tianhang['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_rice',x.split('/')[-1]))
df_tianhang['label'] = df_tianhang['LAI']
df_tianhang = df_tianhang.dropna()
df_tianhang = df_tianhang.reset_index()
df_tianhang.head()

,index,pic_key,date_hour,date,co2,stemp,stemp2,stemp3,stemp4,stemp5,...,pm10,pm25,press,solar,temp,wind_d,wind_sp,LAI,image_path,label
0,282,/789/1655496854_1655496673_4.jpg,2022-06-18 04,2022-06-18,624.0,19.8,19.6,19.8,19.3,19.1,...,6.0,6.0,991.1,2.52,17.26,274.3,3.75,1.626667,/home/junsheng/data/tianhang_rice/1655496854_1...,1.626667
1,283,/789/1655496854_1655496673_4.jpg,2022-06-18 04,2022-06-18,624.0,19.8,19.6,19.8,19.3,19.1,...,7.0,7.0,991.2,5.93,17.18,268.7,2.67,1.626667,/home/junsheng/data/tianhang_rice/1655496854_1...,1.626667
2,284,/789/1655504090_1655503874_4.jpg,2022-06-18 06,2022-06-18,617.0,19.5,19.5,19.6,19.1,19.0,...,5.0,5.0,991.9,8.84,17.75,248.6,2.07,1.626667,/home/junsheng/data/tianhang_rice/1655504090_1...,1.626667
3,285,/789/1655504090_1655503874_4.jpg,2022-06-18 06,2022-06-18,617.0,19.5,19.4,19.5,19.1,19.0,...,3.0,3.0,992.0,9.20,17.83,265.7,2.95,1.626667,/home/junsheng/data/tianhang_rice/1655504090_1...,1.626667
4,286,/789/1655511249_1655511073_4.jpg,2022-06-18 08,2022-06-18,604.0,19.3,19.2,19.4,19.1,18.9,...,1.0,1.0,992.6,17.75,18.98,275.4,3.62,1.626667,/home/junsheng/data/tianhang_rice/1655511249_1...,1.626667


数据检查

In [64]:
# 检查图片下载的全不全
pic = df_tianhang.image_path.map(lambda x:x.split('/')[-1]).unique()
print(len(pic))
file_ls = os.listdir("/home/junsheng/data/tianhang_rice")
print(len(file_ls))
ret = list(set(pic) ^ set(file_ls))
print(ret) #差集
# assert len(pic)==len(file_ls),"请检查下载的图片，缺了{}个".format(len(pic)-len(file_ls))


541
763
['1662156938_1662156676_4.jpg', '1653783194_1653783073_4.jpg', '1654416840_1654416674_4.jpg', '1655259332_1655259076_4.jpg', '1653790398_1653790273_4.jpg', '1654467235_1654467074_4.jpg', '1655201643_1655201475_4.jpg', '1658304910_1658304676_4.jpg', '1657966494_1657966275_4.jpg', '1655460872_1655460672_4.jpg', '1662272091_1662271873_4.jpg', '1655266456_1655266276_4.jpg', '1654755224_1654755073_4.jpg', '1658268849_1658268671_4.jpg', '1653963189_1653963073_4.jpg', '1658283283_1658283073_4.jpg', '1653984798_1653984674_4.jpg', '1662264895_1662264673_4.jpg', '1658384087_1658383873_4.jpg', '1654935249_1654935073_4.jpg', '1662171336_1662171077_4.jpg', '1657829686_1657829472_4.jpg', '1654834532_1654834277_4.jpg', '1654553669_1654553474_4.jpg', '1653876767_1653876672_4.jpg', '1662192998_1662192677_4.jpg', '1654827275_1654827077_4.jpg', '1655280872_1655280676_4.jpg', '1651716813_1651716678_4.jpg', '1654409629_1654409476_4.jpg', '1653804793_1653804673_4.jpg', '1657844100_1657843874_4.jpg',

归一化非object列

In [65]:
list(df_tianhang)

['index',
 'pic_key',
 'date_hour',
 'date',
 'co2',
 'stemp',
 'stemp2',
 'stemp3',
 'stemp4',
 'stemp5',
 'shumi',
 'shumi2',
 'shumi3',
 'shumi4',
 'shumi5',
 'ts',
 'insert_time',
 'humi',
 'pm10',
 'pm25',
 'press',
 'solar',
 'temp',
 'wind_d',
 'wind_sp',
 'LAI',
 'image_path',
 'label']

In [66]:
number_title = []
recorder = {}
for title in df_tianhang:
    # print(df_xiangguan[title].head())
    if title == 'raw_label':
        continue
    if df_tianhang[title].dtype != "object":
        
        number_title.append(title)
        x_min = df_tianhang[title].min()
        x_max = df_tianhang[title].max()
        # print(x_min,x_max)
        recorder[title] = (x_min,x_max)
        df_tianhang[title] = df_tianhang[title].map(lambda x:(x-x_min)/(x_max - x_min))
print(number_title)
print(recorder)

['index', 'co2', 'stemp', 'stemp2', 'stemp3', 'stemp4', 'stemp5', 'shumi', 'shumi2', 'shumi3', 'shumi4', 'shumi5', 'humi', 'pm10', 'pm25', 'press', 'solar', 'temp', 'wind_d', 'wind_sp', 'LAI', 'label']
{'index': (282, 1493), 'co2': (0.0, 1175.0), 'stemp': (13.3, 24.0), 'stemp2': (14.1, 22.8), 'stemp3': (14.1, 23.2), 'stemp4': (14.3, 22.3), 'stemp5': (14.7, 21.8), 'shumi': (73.9, 76.9), 'shumi2': (70.4, 74.8), 'shumi3': (67.5, 69.2), 'shumi4': (72.2, 74.2), 'shumi5': (69.8, 71.8), 'humi': (31.0, 100.0), 'pm10': (0.0, 1333.0), 'pm25': (0.0, 1333.0), 'press': (981.1, 1009.0), 'solar': (0.0, 200.0), 'temp': (7.39, 32.0), 'wind_d': (0.0, 359.8), 'wind_sp': (0.0, 9.41), 'LAI': (1.3458333333333334, 2.2466666666666666), 'label': (1.3458333333333334, 2.2466666666666666)}


In [67]:
# xiangguan_sensor = ['temperature', 'humidity', 'illuminance', 'soil_temperature', 'soil_humidity', 'pressure', 'wind_speed', 'photosynthetic', 'sun_exposure_time', 'COz', 'soil_ph']
tianhang_sensor = ['co2', 'stemp', 'stemp2', 'stemp3', 'stemp4', 'stemp5', 'shumi', 'shumi2', 'shumi3', 'shumi4', 'shumi5', 'humi', 'pm10', 'pm25', 'press', 'solar', 'temp', 'wind_d', 'wind_sp']

df_tianhang['sensor'] = df_tianhang[tianhang_sensor].values.tolist()
print("input dim:",len(tianhang_sensor))

input dim: 19


In [68]:
df=df_tianhang
if config.debug:
    df = df[:100]
df.shape

(1082, 29)

create folds

In [69]:
skf = StratifiedKFold(n_splits=config.n_fold, shuffle=True, random_state=config.seed)  
for fold, (train_idx, val_idx) in enumerate(skf.split(df,df.date)):
    df.loc[val_idx, 'fold'] = fold
df.groupby(['fold'])['label'].count()# ???

fold
0.0    217
1.0    217
2.0    216
3.0    216
4.0    216
Name: label, dtype: int64

In [70]:
df.head()
df.to_csv("test_fold.csv",index=False)

# dataset


In [71]:
myTransforms = transforms.Compose([
    transforms.Resize((config.img_size,config.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.7136, 0.7118, 0.6788],
    std=[0.3338, 0.3453, 0.3020],
    
)
])

def load_img(path):
    img =  Image.open(path).convert('RGB')
    img = myTransforms(img)
    return img

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=True, transforms=None):
        self.df         = df
        self.label      = label
        self.sensors = df['sensor'].tolist()
        self.img_paths  = df['image_path'].tolist()   
        if self.label:
            self.labels = df['label'].tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = self.img_paths[index]
        img = load_img(img_path)
        sensor = self.sensors[index]
        sensor = torch.tensor(sensor).unsqueeze(0) #[1,n]
        if self.label:
            label = self.labels[index]
            return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)
        else:
            return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float)

# dataloader

In [72]:
def fetch_dataloader(fold:int):
    train_df = df.query("fold!=@fold").reset_index(drop=True)

    valid_df = df.query("fold==@fold").reset_index(drop=True)
    print("train_df.shape:",train_df.shape)
    print("valid_df.shape:",valid_df.shape)

    train_data  = BuildDataset(df=train_df,label=True)
    valid_data = BuildDataset(df=valid_df,label=True)

    train_loader = DataLoader(train_data, batch_size=config.train_batch_size,shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=config.valid_batch_size,shuffle=False)
    # test_loader = DataLoader(test_data, batch_size=config.test_batch_size,shuffle=False)
    return train_loader,valid_loader


In [73]:
# train_dataset = BuildDataset(df=df)
# train_loader = DataLoader(train_dataset, batch_size=config.train_batch_size,shuffle=True)
# valid_loader = DataLoader(train_dataset, batch_size=config.valid_batch_size,shuffle=True)
train_loader,valid_loader = fetch_dataloader(fold=0)


train_df.shape: (865, 30)
valid_df.shape: (217, 30)


In [74]:
img,sensor,label = next(iter(train_loader))
print(img.shape)
print(sensor.shape)
print(label.shape)

/tmp/ipykernel_3847427/355586058.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)


torch.Size([32, 3, 384, 384])
torch.Size([32, 1, 19])
torch.Size([32])


# model

sensorViLTransformerSS

In [75]:

class sensorViLTransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device)
       
            (
                image_embeds, # torch.Size([1, 217, 768])
                image_masks, # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        batch_size = img.shape[0]
        sensor_masks = torch.ones(batch_size,1).to(config.device) # 序列数量
        image_masks = image_masks.to(config.device)
        co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])

        x = co_embeds.to(config.device) # torch.Size([1, 211, 768])

        for i, blk in enumerate(self.transformer.blocks): 
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks) # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
            x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
            x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
           "sensor_feats":sensor_feats,
            "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            "image_labels": image_labels,
            "image_masks": image_masks,
           
            "patch_index": patch_index,

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


sensorOnlyViLTransformerSS

In [76]:

class sensorOnlyViLTransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        # mask_image=False,
        # image_token_type_idx=1,
        # image_embeds=None,
        # image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        

        # if image_embeds is None and image_masks is None:
        #     img = batch["image"].to(config.device)
       
        #     (
        #         image_embeds, # torch.Size([1, 217, 768])
        #         image_masks, # torch.Size([1, 217])
        #         patch_index,
        #         image_labels,
        #     ) = self.transformer.visual_embed(
        #         img,
        #         max_image_len=config.max_image_len,
        #         mask_it=mask_image,
        #     )
        # else:
        #     patch_index, image_labels = (
        #         None,
        #         None,
        #     )
        # 用embedding对数据输入预处理，降低维度
        # image_embeds = image_embeds + self.token_type_embeddings(
        #         torch.full_like(image_masks, image_token_type_idx)
        #     )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        # batch_size = img.shape[0]
        sensor_masks = torch.ones(sensor_embeds.shape[1],1).to(config.device) # 序列数量
        # image_masks = image_masks.to(config.device)
        # co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        # co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])
        co_embeds = sensor_embeds
        co_masks = sensor_masks

        x = co_embeds.to(config.device) # torch.Size([1, 1, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        # sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
        #     x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
        #     x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        # )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
        #    "sensor_feats":sensor_feats,
            # "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            # "image_labels": image_labels,
            # "image_masks": image_masks,
           
            # "patch_index": patch_index,

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


## model build

In [77]:
model = sensorOnlyViLTransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
# model = sensorViLTransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
model.to(config.device)
print(config.device)
for i,m in enumerate(model.modules()):
    print(i,m)

No pretrained weights exist or were found for this model. Using random initialization.


cuda:0
0 sensorOnlyViLTransformerSS(
  (sensor_linear): Linear(in_features=19, out_features=768, bias=True)
  (token_type_embeddings): Embedding(2, 768)
  (transformer): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (pos_drop): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.1, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in

test

In [78]:

# sensor = torch.rand(config.senser_input_num)
# # sensor = torch.ones(config.senser_input_num)
# print(sensor)
# sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])
# batch = {}
# batch['sensor'] = sensor
# batch['image'] = "/home/junsheng/data/xiangguan/pic/xiangguanD4-2021-05-24-10-00-25.jpeg"
# model(batch)

# 损失函数

In [79]:
criterion = F.mse_loss #均方误差损失函数
# criterion = F.mae_loss

# train one epoch

In [80]:



def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (img, sensor,label) in pbar:         
        # img = img.to(device, dtype=torch.float)
        # sensor  = sensor.to(device, dtype=torch.float)
        # label  = label.to(device, dtype=torch.float)
        batch_size = img.size(0)
        
        batch = {"image":img,"sensor":sensor}

        y_pred = model(batch)
        label = label.to(config.device).unsqueeze(1)
        loss = criterion(y_pred['cls_output'], label)
        
        #一坨优化
        optimizer.zero_grad()#每一次反向传播之前都要归零梯度
        loss.backward()      #反向传播
        optimizer.step()     #固定写法
        scheduler.step()
     
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')

    
        
        
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

# valid one epoch

In [81]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, optimizer):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (img, sensor,label) in pbar:               
        
        
        batch_size = img.size(0)
        batch = {"image":img,"sensor":sensor}

        y_pred  = model(batch)
        label = label.to(config.device).unsqueeze(1)

        loss = criterion(y_pred['cls_output'], label)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

# train

In [82]:

def run_training(model, optimizer, scheduler, device, num_epochs):
     # init wandb
    run = wandb.init(project="vilt",
                    config={k: v for k, v in dict(vars(config)).items() if '__' not in k},
                    # config={k: v for k, v in dict(config).items() if '__' not in k},
                    anonymous=anonymous,
                    # name=f"vilt|fold-{config.valid_fold}",
                    name=config.wandb_name,
                    # group=config.wandb_group,
                    )
    wandb.watch(model, log_freq=100)

    best_loss = 9999
    best_valid_loss = 9999
    history = defaultdict(list)
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=device, epoch=epoch)
        val_loss = valid_one_epoch(model,valid_loader,device=device,optimizer=optimizer)
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)

        wandb.log({"Train Loss": train_loss,
                    "Valid Loss": val_loss,
                "lr": scheduler.get_last_lr()[0]
                })
        if best_valid_loss > val_loss:
            best_valid_loss = val_loss
            # model_file_path = os.path.join(wandb.run.dir,"epoch-{}-{}.bin".format(epoch,wandb.run.id))
            model_file_path = os.path.join(wandb.run.dir,"epoch-best.bin")
            run.summary["Best Epoch"] = epoch
            torch.save(model.state_dict(), model_file_path)
            print("model save to", model_file_path)
            
    os.system("cp /home/junsheng/ViLT/my_vilt_tianhang_rice.ipynb {}".format(wandb.run.dir))
    run.finish()
    return model, history

optimizer

In [83]:
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=config.T_max, 
                                                   eta_min=1e-5)


run train

In [84]:

model, history = run_training(model, optimizer, scheduler,device=config.device,num_epochs=config.max_epoch)



cuda: NVIDIA GeForce RTX 3090

Epoch 1/50

Train :   0%|          | 0/28 [00:00<?, ?it/s]/tmp/ipykernel_3847427/355586058.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)
Valid : 100%|██████████| 55/55 [00:12<00:00,  4.35it/s, gpu_memory=2.36 GB, lr=0.00100, valid_loss=0.2307]


model save to /home/junsheng/ViLT/wandb/offline-run-20220926_204142-13q6ro4s/files/epoch-best.bin
Epoch 2/50

Train :  14%|█▍        | 4/28 [00:08<00:53,  2.23s/it, gpu_mem=2.38 GB, lr=0.00100, train_loss=0.2576]


KeyboardInterrupt: 

# infer

In [ ]:
for (img,sensor,label) in valid_loader:
    print(img.shape,sensor.shape,label)
    break

torch.Size([4, 3, 384, 384]) torch.Size([4, 1, 11]) tensor([0., 0., 0., 0.])


/tmp/ipykernel_446673/355586058.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)


In [54]:
# torch.save(model.state_dict(), 'embedding_test_dict.pt')
# print(model)

# model.load_state_dict(torch.load("/home/junsheng/ViLT/wandb/offline-run-20220811_120519-nzfb1xoz/files/epoch-best.bin"))
model.eval()
device = config.device
model.to(device)
def infer(img_filename, sensor):
    try:
        img_path = os.path.join('pictures',img_filename)
        image = Image.open(img_path).convert("RGB")
        img = pixelbert_transform(size=384)(image) # 将图像数据归一化torch.Size([3, 384, 576])
        img = torch.tensor(img)
        img = torch.unsqueeze(img, 0) # torch.Size([1, 3, 384, 576])
        img = img.to(device)
        print("img.shape:",img.shape)
    except :
        print("图片加载失败！")
        raise

    batch = dict()
    batch["image"] = img

    batch['sensor_masks'] = torch.ones(1,1).to(device)
    with torch.no_grad():
        batch['sensor'] = sensor.to(device)       
        infer = model(batch)

        print(infer)
        sensor_emb, img_emb = infer["sensor_feats"], infer["image_feats"]# torch.Size([1, 23, 768]) torch.Size([1, 217, 768])
        cls_output = infer['cls_output']
        

    return [cls_output]


random test

In [55]:

examples=[
            "/home/junsheng/data/xiangguan/pic/xiangguanD4-2021-05-24-10-00-25.jpeg", #0
            
            "/home/junsheng/data/xiangguan/pic/xiangguanD4-2021-07-18-04-22-30-preset-18.jpeg", # 3
    ]



n = 1
sensor = torch.rand(config.senser_input_num)
# sensor = torch.ones(config.senser_input_num)
print(sensor)
sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])
out = infer(examples[0],sensor)
# print("out:",out,"000\n")
# print("out0.shape:",out[0].shape)
# cv2.imwrite('output.png',out[0])



tensor([0.1375, 0.8209, 0.0185, 0.3641, 0.6405, 0.4498, 0.3304, 0.3734, 0.0022,
        0.7310, 0.2874, 0.9734, 0.3282, 0.9755, 0.5969, 0.8699, 0.7631, 0.9917,
        0.4927])
img.shape: torch.Size([1, 3, 352, 608])


/tmp/ipykernel_3847427/3499233738.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])


{'sensor_feats': tensor([[[ 3.0098e-06, -1.7633e-07,  1.6960e-07, -1.6755e-08, -2.2477e-06,
           3.1875e-06,  6.5734e-06, -4.7667e-09, -3.6099e-07, -7.4075e-07,
          -1.5870e-07,  3.6547e-06, -3.8343e-08, -9.8189e-06, -1.4985e-08,
          -1.8155e-07, -2.6160e-06,  9.1580e-09, -2.9543e-06, -1.4204e-08,
           9.1015e-07,  9.1730e-10,  2.2748e-08, -1.0482e-08,  5.9382e-07,
           3.1595e-06, -2.8178e-06, -7.7881e-10,  2.9493e-06,  4.5215e-06,
          -9.4157e-06,  1.0949e-06, -7.7766e-07,  1.1918e-08, -1.0499e-06,
           1.1121e-06,  2.0103e-07, -7.6055e-08, -2.8639e-09,  2.2348e-06,
           7.2907e-08,  8.4753e-06,  3.4430e-08, -2.6252e-07, -4.6850e-06,
           1.4032e-08,  7.8041e-08, -8.3715e-08, -2.1338e-06, -2.3601e-06,
           1.0706e-07, -4.3645e-06, -3.1781e-08, -1.7366e-09, -2.4237e-06,
          -1.8145e-09, -1.0051e-05,  3.1867e-06,  3.3794e-07, -5.3987e-08,
           1.0769e-05, -2.3083e-07, -3.7618e-07,  2.0922e-08, -7.3564e-08,
        

/home/junsheng/.conda/envs/pytorch_junsheng_39/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [56]:
out

[tensor([[0.4540]], device='cuda:0')]

In [58]:
print(out[0].cpu().numpy()[0][0])
#0.00031266143

0.4539965


test by valid

选择三组生长期不同的数据去验证训练的结果

In [51]:
df_test = df.query("fold==0").reset_index(drop=True)
df_test.to_csv("test_by_valid.csv",index=False)
sensor_test_list = df_test.sensor.tolist()
image_test_list = df_test.image_path.tolist()

In [52]:
idx = 64
sensor =  torch.tensor(sensor_test_list[idx]).unsqueeze(0).unsqueeze(0)
out = infer(image_test_list[idx],sensor)

NameError: name 'infer' is not defined

In [ ]:
idx = 876
sensor =  torch.tensor(sensor_test_list[idx]).unsqueeze(0).unsqueeze(0)
out = infer(image_test_list[idx],sensor)

img.shape: torch.Size([1, 3, 352, 608])
{'sensor_feats': tensor([[[ 3.0827e-06, -1.7530e-07,  1.5452e-07, -2.2674e-08, -2.2496e-06,
           3.2150e-06,  6.0465e-06, -5.3801e-09, -1.8847e-07, -7.7690e-07,
          -1.5788e-07,  3.6213e-06, -3.8558e-08, -1.0063e-05, -1.4680e-08,
          -1.8126e-07, -2.7730e-06,  9.1713e-09, -2.7553e-06, -5.3852e-09,
           9.5103e-07,  3.8168e-09,  7.2278e-08, -1.0654e-08,  6.3138e-07,
           3.1689e-06, -2.6612e-06, -1.0689e-09,  2.8926e-06,  4.4980e-06,
          -9.2651e-06,  1.0652e-06, -4.1821e-07,  2.2096e-08, -9.7153e-07,
           1.0851e-06,  7.0076e-08, -7.6146e-08, -2.9167e-09,  2.1692e-06,
           7.3493e-08,  8.4667e-06,  3.4533e-08, -2.6099e-07, -4.6151e-06,
           1.3888e-08,  3.3025e-07, -5.3568e-06, -2.0778e-06, -2.7470e-06,
           1.0874e-07, -4.3026e-06, -3.1775e-08, -1.3380e-09, -2.3559e-06,
          -1.7084e-09, -9.4255e-06,  2.6611e-06,  7.8697e-07, -5.3748e-08,
           1.0649e-05, -2.6270e-07, -3.8041

/tmp/ipykernel_446673/164085027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img)


In [ ]:
idx = 1817
sensor =  torch.tensor(sensor_test_list[idx]).unsqueeze(0).unsqueeze(0)
out = infer(image_test_list[idx],sensor)

img.shape: torch.Size([1, 3, 352, 608])
{'sensor_feats': tensor([[[ 3.0705e-06, -1.7521e-07,  1.5456e-07, -2.1202e-08, -2.2476e-06,
           3.2173e-06,  6.0811e-06, -5.3703e-09, -1.9807e-07, -7.9415e-07,
          -1.5692e-07,  3.6335e-06, -3.8573e-08, -1.0093e-05, -1.3580e-08,
          -1.8139e-07, -2.7724e-06,  9.1703e-09, -2.7452e-06, -5.7367e-09,
           9.5353e-07,  3.9170e-09,  8.3201e-08, -1.0664e-08,  6.3288e-07,
           3.1788e-06, -2.6123e-06, -1.0732e-09,  2.9125e-06,  4.4989e-06,
          -9.3056e-06,  1.0643e-06, -4.5918e-07,  2.2646e-08, -9.7590e-07,
           1.0921e-06,  5.4508e-08, -7.6128e-08, -2.9861e-09,  2.1609e-06,
           7.3610e-08,  8.4742e-06,  3.4496e-08, -2.6085e-07, -4.6524e-06,
           1.3929e-08, -5.0008e-07, -2.3619e-06, -2.0132e-06, -2.7828e-06,
           1.0898e-07, -4.3159e-06, -3.1896e-08, -1.0233e-09, -2.3536e-06,
          -1.7547e-09, -9.5498e-06,  2.6606e-06,  2.5922e-07, -5.3656e-08,
           1.0648e-05, -2.6302e-07, -3.8063

/tmp/ipykernel_446673/164085027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img)
